In [ ]:
# !mkdir input
# !mkdir output

In [ ]:
# !pip install bitsandbytes==0.42.0
# !pip install git+https://github.com/shumingma/transformers.git

  Using cached bitsandbytes-0.42.0-py3-none-any.whl.metadata (9.9 kB)
Using cached bitsandbytes-0.42.0-py3-none-any.whl (105.0 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.39.0
    Uninstalling bitsandbytes-0.39.0:
      Successfully uninstalled bitsandbytes-0.39.0


In [ ]:
!python3 kvcache.py --kvcache file  --output "./output/result_1.txt"

2025-04-19 07:18:42.539428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745047122.559753    4272 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745047122.565812    4272 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-19 07:18:42.585542: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-19 07:18:50,662 - INFO - Using device: cuda
False

===================================BUG REPORT=============

In [ ]:
# easy way

In [ ]:
!pip install git+https://github.com/shumingma/transformers.git

  Cloning https://github.com/shumingma/transformers.git to /tmp/pip-req-build-3lhe6ob8
  Running command git clone --filter=blob:none --quiet https://github.com/shumingma/transformers.git /tmp/pip-req-build-3lhe6ob8
  Resolved https://github.com/shumingma/transformers.git to commit 4a01efe84d0120dc2545ff2de445082400d87407
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.52.0.dev0-py3-none-any.whl size=11413940 sha256=ca097c6a7ecc292a47a675e56bbeeae941c5aa184397a2c0b738eb0ae92781ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-mtwfga5v/wheels/2c/2a/7c/3be0c30fb51a7becc4bcbb536739ae9ed9cc7e633ffbfaf63b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.cache_utils import DynamicCache
# import os

In [ ]:
input_file_path =  "hp4.txt"
model_name = "unsloth/Llama-3.2-1B-Instruct"
model_name = "unsloth/Qwen2.5-3B-Instruct"

In [ ]:
# Minimal generate function for token-by-token generation
def generate(model, input_ids: torch.Tensor, past_key_values, max_new_tokens: int = 50) -> torch.Tensor:
    device = model.model.embed_tokens.weight.device
    origin_len = input_ids.shape[-1]
    input_ids = input_ids.to(device)
    output_ids = input_ids.clone()
    next_token = input_ids

    with torch.no_grad():
        for _ in range(max_new_tokens):
            out = model(
                input_ids=next_token,
                past_key_values=past_key_values,
                use_cache=True
            )
            logits = out.logits[:, -1, :]
            token = torch.argmax(logits, dim=-1, keepdim=True)
            output_ids = torch.cat([output_ids, token], dim=-1)
            past_key_values = out.past_key_values
            next_token = token.to(device)

            if model.config.eos_token_id is not None and token.item() == model.config.eos_token_id:
                break

    # Return just the newly generated part
    return output_ids[:, origin_len:]

In [ ]:
torch.serialization.add_safe_globals([DynamicCache])
torch.serialization.add_safe_globals([set])

def get_kv_cache(model, tokenizer, prompt: str) -> DynamicCache:
    # Encode prompt
    device = model.model.embed_tokens.weight.device
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    cache = DynamicCache()

    with torch.no_grad():
        _ = model(
            input_ids=input_ids,
            past_key_values=cache,
            use_cache=True
        )
    return cache

def clean_up(cache: DynamicCache, origin_len: int):
    # Remove any tokens appended to the original knowledge
    for i in range(len(cache.key_cache)):
        cache.key_cache[i] = cache.key_cache[i][:, :, :origin_len, :]
        cache.value_cache[i] = cache.value_cache[i][:, :, :origin_len, :]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
import torch._dynamo
torch._dynamo.config.suppress_errors = True
model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="cuda", # <==== TODO change this
        )

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu" # TODO change this to cuda
model.to(device)
print(f"Loaded {model_name}.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded unsloth/Qwen2.5-3B-Instruct.


In [ ]:
%%time
with open(input_file_path, "r") as f:
    doc_text = f.read()

system_prompt = f"""
<|system|>
You are an assistant who provides concise factual answers.
<|user|>
Context:
{doc_text[:int(len(doc_text)/8)]}
""".strip()

# Build the cache
doc_cache = get_kv_cache(model, tokenizer, system_prompt)
origin_len = doc_cache.key_cache[0].shape[-2]
print("KV cache built.")

Token indices sequence length is longer than the specified maximum sequence length for this model (33249 > 32768). Running this sequence through the model will result in indexing errors


OutOfMemoryError: CUDA out of memory. Tried to allocate 9.41 GiB. GPU 0 has a total capacity of 14.74 GiB of which 6.15 GiB is free. Process 107080 has 8.59 GiB memory in use. Of the allocated memory 7.03 GiB is allocated by PyTorch, and 1.44 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
#TODO save the cache and invoke it for inferencing
import pickle
cache_path = 'doc_cache_hp_1.pkl'
with open(cache_path, 'wb') as file:
    pickle.dump(doc_cache, file)

NameError: name 'doc_cache' is not defined

In [ ]:
%%time
# 1st query
question1 = "What habit did the boys from village had?"
clean_up(doc_cache, origin_len)
input_ids_q1 = tokenizer(question1 + "\n", return_tensors="pt").input_ids.to(device) # TODO change this to cuda
gen_ids_q1 = generate(model, input_ids_q1, doc_cache)
answer1 = tokenizer.decode(gen_ids_q1[0], skip_special_tokens=True)
print("Q1:", question1)
print("A1:", answer1)

Q1: What habit did the boys from village had?
A1: "Ah, yes, Leprechauns!" said Harry, grinning.  "I knew it!  I knew it!"
The crowd erupted into cheers and applause.  Harry grinned back at the shamrock, feeling a strange sense of pride
CPU times: user 6.82 s, sys: 37.3 ms, total: 6.86 s
Wall time: 6.98 s


In [ ]:
int(len(doc_text)/8)

138647